Codigo de ejecucion del codigo de redes opticas

In [3]:
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.utils import set_random_seed
from custom_env.redes_opticas_env import RedesOpticasEnv  # Asegúrate de que esta ruta sea correcta

def make_env(env_id: str, rank: int, seed: int):
    def _init():
        try:
            env = gym.make(env_id, render_mode=None)  # Cambia a None si no quieres modo visual
            env.reset(seed=seed + rank)
            return env
        except Exception as e:
            print(f"Error al inicializar el entorno: {e}")
            raise e
    set_random_seed(seed)
    return _init

if __name__ == "__main__":
    env_id = 'RedesOpticasEnv-v0'  # Asegúrate de que este ID coincida con el registrado
    num_cpu = 4  # Reduce el número si es necesario
    seed = np.random.randint(0, 10)
    vec_env = DummyVecEnv([make_env(env_id, i, seed) for i in range(num_cpu)])
    model = PPO("MultiInputPolicy", vec_env, verbose=1, n_steps=2048)
    model.learn(total_timesteps=100_000)

    # Fase de prueba
    num_test_episodes = 10  # Número de episodios de prueba
    episode_rewards = []  # Lista para guardar las recompensas totales de cada episodio de prueba

    for episode in range(num_test_episodes):
        obs = vec_env.reset()  # Resetea el entorno al estado inicial
        done = np.array([False]*num_cpu)  # Inicializa 'done' para todos los entornos
        total_rewards = np.array([0.0]*num_cpu)  # Inicializa la recompensa total por entorno
        
        while not done.all():
            action, _states = model.predict(obs, deterministic=True)  # Usa el modelo para predecir la acción
            obs, rewards, dones, info = vec_env.step(action)  # Ejecuta la acción en el entorno
            total_rewards += rewards  # Acumula las recompensas
            done |= dones 
        
        episode_rewards.append(total_rewards)  # Guarda las recompensas totales de este episodio

    # Análisis de los resultados de la prueba
    print(episode_rewards)
    recompensa_promedio = np.mean(episode_rewards)

    print(f"Recompensa promedio en los episodios de prueba: {recompensa_promedio}")
    objetivo_deseado = 500  # En nuestro caso el objetivo es acercarnos al Balloc = 500
    if recompensa_promedio >= objetivo_deseado:
        print("¡Objetivo alcanzado!")
    else:
        print("Objetivo no alcanzado.")


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 4724 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 8192 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1966         |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0007170341 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -14.2        |
|    explained_variance   | 1.19e-07     |
|    learning_rate        | 0.0003       |
|    loss                 | 3.3e+09      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000765    |
|    std                  | 1            |
|    value_loss           | 6.47e+09     |